<a href="https://colab.research.google.com/github/crystalloide/RAG/blob/main/LAB25_Agent_avec_m%C3%A9moire_%C3%A9pisodique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB25 : Implémentation d'un agent avec mémoire épisodique :

**Objectif:** Capturer des "épisodes" d'interaction (qui/quoi/quand), les stocker avec des embeddings, et récupérer les plus pertinents pour enrichir le contexte de l'agent.

**Durée estimée:** 25–30 minutes

**Livrable:** Notebook qui met en oeuvre un agent enregistre des épisodes, les récupère, et les utilise pour répondre aux questions.

---

## Table des matières
1. Installation & Configuration de l'environnement (5 min)
2. Définition du store de mémoire épisodique avec Chroma (2 min)
3. Scoring recence–similarité (5 min)
4. Agent simple qui enregistre les épisodes (5 min)
5. Seed et tests (5 min)
6. Enrichissement avec rôles & tags (optionnel, 5 min)
7. Intégration future dans des agents avancés

---
## 1. Installation & Configuration de l'environnement
**Durée:** 5 minutes

Installation des dépendances nécessaires pour OpenAI, ChromaDB et gestion des variables d'environnement.

In [1]:
# Installation des packages
!pip install -q openai chromadb python-dotenv

print("✅ Packages installés avec succès")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.

### Configuration de la clé API OpenAI

**Deux options:**
- **Option A (recommandée pour Colab):** Utiliser les secrets de Colab
- **Option B:** Entrer directement la clé (⚠️ attention à ne pas partager le notebook)

In [2]:
import os
from getpass import getpass

# Option A: Utiliser les secrets de Google Colab (recommandé)
# Dans le menu de gauche: 🔑 Secrets → Ajouter OPENAI_API_KEY
try:
    from google.colab import userdata
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    print("✅ Clé API chargée depuis les secrets Colab")
except:
    # Option B: Saisie manuelle (la clé ne sera pas affichée)
    os.environ['OPENAI_API_KEY'] = getpass("Entrez votre clé API OpenAI: ")
    print("✅ Clé API configurée manuellement")

# Vérification
if not os.environ.get('OPENAI_API_KEY'):
    raise ValueError("❌ OPENAI_API_KEY n'est pas définie!")
else:
    print(f"✅ Clé API présente (longueur: {len(os.environ['OPENAI_API_KEY'])} caractères)")

✅ Clé API chargée depuis les secrets Colab
✅ Clé API présente (longueur: 164 caractères)


---
## 2. Définition du stockage de la mémoire épisodique (Chroma)
**Durée:** 2 minutes

Mise en place de ChromaDB pour stocker les épisodes avec leurs embeddings et métadonnées.

In [3]:
import time
import uuid
import math
from datetime import datetime, timezone
from openai import OpenAI
import chromadb

# Initialisation du client OpenAI
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

# Modèle d'embedding à utiliser
EMB_MODEL = "text-embedding-3-small"

print(f"✅ Client OpenAI initialisé")
print(f"📊 Modèle d'embedding: {EMB_MODEL}")

✅ Client OpenAI initialisé
📊 Modèle d'embedding: text-embedding-3-small


In [4]:
def embed(text: str):
    """Génère un embedding pour le texte donné."""
    response = client.embeddings.create(model=EMB_MODEL, input=text)
    return response.data[0].embedding

# Test de la fonction d'embedding
test_embedding = embed("Test de génération d'embedding")
print(f"✅ Fonction embed() opérationnelle")
print(f"📏 Dimension de l'embedding: {len(test_embedding)}")

✅ Fonction embed() opérationnelle
📏 Dimension de l'embedding: 1536


In [5]:
# Création d'une collection Chroma locale pour les épisodes
chroma = chromadb.Client()
episodes = chroma.get_or_create_collection(name="episodic_memory")

print(f"✅ Collection ChromaDB créée/récupérée")
print(f"📦 Nom de la collection: episodic_memory")
print(f"🔢 Nombre d'épisodes actuels: {episodes.count()}")

✅ Collection ChromaDB créée/récupérée
📦 Nom de la collection: episodic_memory
🔢 Nombre d'épisodes actuels: 0


In [6]:
def now_ts():
    """Retourne le timestamp actuel en secondes."""
    return int(time.time())

def ts_to_str(ts: int):
    """Convertit un timestamp en chaîne de caractères lisible."""
    return datetime.fromtimestamp(ts, tz=timezone.utc).strftime("%Y-%m-%d %H:%M:%S UTC")

def add_episode(summary: str, who: str = "user", tags=None):
    """Stocke un épisode avec un résumé textuel + métadonnées."""
    eid = str(uuid.uuid4())
    meta = {
        "who": who,
        "summary": summary,
        "ts": now_ts(),
        "tags": ",".join(tags) if tags else ""
    }
    episodes.add(
        ids=[eid],
        embeddings=[embed(summary)],
        metadatas=[meta],
        documents=[summary]
    )
    return eid

# Test de la fonction d'ajout
test_id = add_episode(
    summary="Épisode de test pour vérifier le fonctionnement du système",
    who="system",
    tags=["test", "init"]
)
print(f"✅ Fonction add_episode() opérationnelle")
print(f"🆔 ID de l'épisode test: {test_id}")
print(f"🔢 Nombre total d'épisodes: {episodes.count()}")

✅ Fonction add_episode() opérationnelle
🆔 ID de l'épisode test: d4018dcd-f9d3-46ad-a68d-f2e59a7297e1
🔢 Nombre total d'épisodes: 1


---
## 3. Scoring Fraîcheur–Similarité
**Durée:** 5 minutes

Implémentation d'un score hybride combinant la similarité sémantique et la fraîcheur temporelle (ou "récence") :

**Formule:** `score = α·similarity + (1–α)·recency_weight`

où `recency_weight = exp(-Δt/τ)`

- **α** (alpha): pondération entre similarité et récence (0.7 par défaut = 70% similarité, 30% récence)
- **τ** (tau): temps de demi-vie en heures (72h par défaut = 3 jours)

In [7]:
def search_episodes(query: str, k: int = 5, alpha: float = 0.7, tau_hours: float = 72):
    """
    Retourne les top-k épisodes par score hybride similarité + récence.

    Args:
        query: Texte de la requête
        k: Nombre d'épisodes à retourner
        alpha: Pondération similarité (0-1, défaut 0.7)
        tau_hours: Temps de demi-vie pour la récence en heures (défaut 72h)

    Returns:
        Liste de dictionnaires contenant les épisodes et leurs scores
    """
    qemb = embed(query)

    # Recherche par similarité brute (on récupère plus de résultats pour le re-ranking)
    res = episodes.query(
        query_embeddings=[qemb],
        n_results=min(20, episodes.count()),
        include=["metadatas", "distances", "documents", "embeddings"]
    )

    if not res["ids"] or len(res["ids"][0]) == 0:
        return []

    now = now_ts()
    tau = tau_hours * 3600.0  # conversion en secondes
    out = []

    for i in range(len(res["ids"][0])):
        meta = res["metadatas"][0][i]
        doc = res["documents"][0][i]

        # Chroma retourne une distance; conversion en similarité
        distance = res["distances"][0][i] if res["distances"] else 0.0
        similarity = 1 - distance  # inversion simple pour la démo

        # Calcul de la récence
        dt = max(0, now - int(meta["ts"]))
        rec = math.exp(-dt / tau)

        # Score hybride
        blended = alpha * similarity + (1 - alpha) * rec

        out.append({
            "id": res["ids"][0][i],
            "summary": doc,
            "who": meta["who"],
            "ts": meta["ts"],
            "when": ts_to_str(int(meta["ts"])),
            "similarity": round(similarity, 4),
            "recency": round(rec, 4),
            "score": round(blended, 4),
            "tags": meta.get("tags", "")
        })

    # Tri par score décroissant
    out.sort(key=lambda x: x["score"], reverse=True)
    return out[:k]

print("✅ Fonction search_episodes() définie")
print("📊 Paramètres par défaut:")
print("   - k = 5 (nombre de résultats)")
print("   - α = 0.7 (70% similarité, 30% récence)")
print("   - τ = 72h (temps de demi-vie)")

✅ Fonction search_episodes() définie
📊 Paramètres par défaut:
   - k = 5 (nombre de résultats)
   - α = 0.7 (70% similarité, 30% récence)
   - τ = 72h (temps de demi-vie)


### Test de la fonction de recherche

In [8]:
# Test de recherche sur l'épisode de test créé précédemment
test_results = search_episodes("système de test", k=3)

print("🔍 Résultats de recherche pour 'système de test':")
print(f"📊 Nombre de résultats: {len(test_results)}\n")

for i, result in enumerate(test_results, 1):
    print(f"\n--- Résultat {i} ---")
    print(f"📝 Résumé: {result['summary'][:80]}...")
    print(f"👤 Qui: {result['who']}")
    print(f"📅 Quand: {result['when']}")
    print(f"🎯 Score total: {result['score']}")
    print(f"   - Similarité: {result['similarity']}")
    print(f"   - Récence: {result['recency']}")
    print(f"🏷️  Tags: {result['tags'] if result['tags'] else 'Aucun'}")

🔍 Résultats de recherche pour 'système de test':
📊 Nombre de résultats: 1


--- Résultat 1 ---
📝 Résumé: Épisode de test pour vérifier le fonctionnement du système...
👤 Qui: system
📅 Quand: 2026-01-30 11:09:57 UTC
🎯 Score total: 0.5889
   - Similarité: 0.4129
   - Récence: 0.9994
🏷️  Tags: test,init


---
## 4. Agent simple qui enregistre les épisodes
**Durée:** 5 minutes

L'agent:
1. Prend une question de l'utilisateur
2. Récupère les mémoires épisodiques les plus pertinentes
3. Incorpore ces mémoires dans le contexte système
4. Génère une réponse
5. Enregistre l'interaction comme un nouvel épisode

In [9]:
def llm_chat(system: str, user: str, model: str = "gpt-4o-mini"):
    """Appel simple au LLM avec contexte système et message utilisateur."""
    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": user}
        ],
        temperature=0.7
    )
    return resp.choices[0].message.content

def format_memories(mem_list):
    """Formate une liste d'informations mémorisées pour inclusion dans le prompt."""
    if not mem_list:
        return "Aucune"

    lines = []
    for m in mem_list:
        lines.append(f"- ({m['when']}) [{m['who']}] {m['summary']}")
    return "\n".join(lines)

def agent_respond(user_text: str, k: int = 3, alpha: float = 0.7, tau_hours: float = 72, verbose: bool = True):
    """
    Réponse de l'agent avec mémoire épisodique.

    Args:
        user_text: Question/message de l'utilisateur
        k: Nombre de mémoires à récupérer
        alpha: Pondération similarité/récence
        tau_hours: Temps de demi-vie pour la récence
        verbose: Afficher les détails de traitement

    Returns:
        Tuple (réponse, mémoires_utilisées)
    """
    if verbose:
        print("\n" + "="*80)
        print("🤖 TRAITEMENT DE LA REQUÊTE")
        print("="*80)

    # 1) Récupérer les épisodes pertinents
    if verbose:
        print(f"\n🔍 Recherche de {k} épisodes pertinents...")

    mems = search_episodes(user_text, k=k, alpha=alpha, tau_hours=tau_hours)
    mem_context = format_memories(mems)

    if verbose:
        print(f"✅ {len(mems)} épisode(s) récupéré(s)\n")
        if mems:
            print("📚 Mémoires utilisées:")
            for i, m in enumerate(mems, 1):
                print(f"  {i}. [{m['who']}] {m['summary'][:60]}... (score: {m['score']})")

    # 2) Construire le prompt système avec contexte mémoire
    system = (
        "Tu es un assistant avec une mémoire épisodique. "
        "Si les 'Épisodes pertinents' contiennent du contexte utile, utilise-le pour informer ta réponse. "
        "Sois concis et cite les faits dans la mesure où ils apparaissent dans les épisodes. "
        "Réponds en français de manière naturelle et professionnelle."
        f"\n\nÉpisodes pertinents:\n{mem_context}"
    )

    # 3) Générer la réponse
    if verbose:
        print("\n💭 Génération de la réponse...")

    answer = llm_chat(system, user_text)

    # 4) Enregistrer cette interaction comme un nouvel épisode
    episode_summary = f"Q: {user_text} | A: {answer[:250]}"
    if len(answer) > 250:
        episode_summary += "..."

    add_episode(summary=episode_summary, who="agent", tags=["dialog", "qa"])

    if verbose:
        print("✅ Réponse générée et enregistrée comme épisode\n")

    return answer, mems

print("✅ Agent avec mémoire épisodique configuré")

✅ Agent avec mémoire épisodique configuré


---
## 5. Alimentation et Tests
**Durée:** 5 minutes

Ajout de quelques épisodes de "vie passée" ou préférences (le genre de choses dont un assistant devrait se souvenir plus tard).

### 5.1 Initialisation de la mémoire avec des épisodes

In [10]:
print("🌱 Initialisation de la mémoire avec des épisodes de démo...\n")

# Épisode 1: Préférence framework
eid1 = add_episode(
    "L'utilisateur a dit que son framework préféré est LangChain et qu'il construit des agents pour l'éducation.",
    who="user",
    tags=["pref", "framework", "langchain", "education"]
)
print(f"✅ Épisode 1 ajouté: Préférence LangChain (ID: {eid1[:8]}...)")

# Épisode 2: Intégration infrastructure
eid2 = add_episode(
    "Nous avons intégré Pinecone hier pour accélérer la recherche sémantique.",
    who="agent",
    tags=["project", "infra", "pinecone", "vector-db"]
)
print(f"✅ Épisode 2 ajouté: Intégration Pinecone (ID: {eid2[:8]}...)")

# Épisode 3: Discussion technique
eid3 = add_episode(
    "L'utilisateur a posé des questions sur les boucles de planification (planning loops) vs les boucles réactives (reactive loops) dans les agents.",
    who="user",
    tags=["topic", "planning", "architecture", "agents"]
)
print(f"✅ Épisode 3 ajouté: Discussion planning loops (ID: {eid3[:8]}...)")

# Épisode 4: Contexte projet
eid4 = add_episode(
    "L'utilisateur travaille sur un projet de RAG pour l'éducation avec ChromaDB et OpenAI.",
    who="user",
    tags=["project", "rag", "education", "chromadb"]
)
print(f"✅ Épisode 4 ajouté: Projet RAG éducation (ID: {eid4[:8]}...)")

# Épisode 5: Préférence modèle
eid5 = add_episode(
    "L'utilisateur préfère utiliser GPT-4o-mini pour un bon équilibre coût/performance dans ses expérimentations.",
    who="user",
    tags=["pref", "model", "openai", "cost-optimization"]
)
print(f"✅ Épisode 5 ajouté: Préférence GPT-4o-mini (ID: {eid5[:8]}...)")

print(f"\n🎉 {episodes.count()} épisodes au total dans la mémoire")

🌱 Initialisation de la mémoire avec des épisodes de démo...

✅ Épisode 1 ajouté: Préférence LangChain (ID: 2b2bed46...)
✅ Épisode 2 ajouté: Intégration Pinecone (ID: 4297d173...)
✅ Épisode 3 ajouté: Discussion planning loops (ID: 77daafa8...)
✅ Épisode 4 ajouté: Projet RAG éducation (ID: 50b17ec5...)
✅ Épisode 5 ajouté: Préférence GPT-4o-mini (ID: e5bb1fd0...)

🎉 6 épisodes au total dans la mémoire


### 5.2 Test 1: Question composite faisant appel plusieurs épisodes mémorisés

In [11]:
question1 = "Quel framework ai-je dit que j'aimais, et qu'avons-nous intégré hier ?"

print("\n" + "#"*80)
print("TEST 1: Question composite")
print("#"*80)
print(f"\n❓ Question: {question1}\n")

answer1, used_mems1 = agent_respond(question1)

print("\n" + "-"*80)
print("💬 RÉPONSE DE L'AGENT")
print("-"*80)
print(answer1)

print("\n" + "-"*80)
print("📚 MÉMOIRES UTILISÉES")
print("-"*80)
for i, m in enumerate(used_mems1, 1):
    print(f"\n{i}. {m['when']} :: {m['summary']}")
    print(f"   Score: {m['score']} (similarité: {m['similarity']}, récence: {m['recency']})")
    print(f"   Tags: {m['tags']}")


################################################################################
TEST 1: Question composite
################################################################################

❓ Question: Quel framework ai-je dit que j'aimais, et qu'avons-nous intégré hier ?


🤖 TRAITEMENT DE LA REQUÊTE

🔍 Recherche de 3 épisodes pertinents...
✅ 3 épisode(s) récupéré(s)

📚 Mémoires utilisées:
  1. [user] L'utilisateur a dit que son framework préféré est LangChain ... (score: 0.3015)
  2. [agent] Nous avons intégré Pinecone hier pour accélérer la recherche... (score: 0.0892)
  3. [user] L'utilisateur a posé des questions sur les boucles de planif... (score: 0.0735)

💭 Génération de la réponse...
✅ Réponse générée et enregistrée comme épisode


--------------------------------------------------------------------------------
💬 RÉPONSE DE L'AGENT
--------------------------------------------------------------------------------
Vous avez dit que votre framework préféré est LangChain, et nous a

### 5.3 Test 2: Rappel d'une discussion technique

In [12]:
question2 = "Rappelle-moi ce que nous avons discuté à propos des boucles de planification vs réactives."

print("\n" + "#"*80)
print("TEST 2: Rappel discussion technique")
print("#"*80)
print(f"\n❓ Question: {question2}\n")

answer2, used_mems2 = agent_respond(question2)

print("\n" + "-"*80)
print("💬 RÉPONSE DE L'AGENT")
print("-"*80)
print(answer2)

print("\n" + "-"*80)
print("📚 MÉMOIRES UTILISÉES")
print("-"*80)
for i, m in enumerate(used_mems2, 1):
    print(f"\n{i}. {m['when']} :: {m['summary']}")
    print(f"   Score: {m['score']} (similarité: {m['similarity']}, récence: {m['recency']})")


################################################################################
TEST 2: Rappel discussion technique
################################################################################

❓ Question: Rappelle-moi ce que nous avons discuté à propos des boucles de planification vs réactives.


🤖 TRAITEMENT DE LA REQUÊTE

🔍 Recherche de 3 épisodes pertinents...
✅ 3 épisode(s) récupéré(s)

📚 Mémoires utilisées:
  1. [user] L'utilisateur a posé des questions sur les boucles de planif... (score: 0.5866)
  2. [agent] Q: Quel framework ai-je dit que j'aimais, et qu'avons-nous i... (score: 0.1933)
  3. [user] L'utilisateur a dit que son framework préféré est LangChain ... (score: 0.0343)

💭 Génération de la réponse...
✅ Réponse générée et enregistrée comme épisode


--------------------------------------------------------------------------------
💬 RÉPONSE DE L'AGENT
--------------------------------------------------------------------------------
Vous avez posé des questions sur les 

### 5.4 Test 3: Vérification infrastructure

In [13]:
question3 = "Avons-nous déjà une base de données vectorielle configurée ?"

print("\n" + "#"*80)
print("TEST 3: Vérification infrastructure")
print("#"*80)
print(f"\n❓ Question: {question3}\n")

answer3, used_mems3 = agent_respond(question3)

print("\n" + "-"*80)
print("💬 RÉPONSE DE L'AGENT")
print("-"*80)
print(answer3)

print("\n" + "-"*80)
print("📚 MÉMOIRES UTILISÉES")
print("-"*80)
for i, m in enumerate(used_mems3, 1):
    print(f"\n{i}. {m['when']} :: {m['summary']}")
    print(f"   Score: {m['score']}")


################################################################################
TEST 3: Vérification infrastructure
################################################################################

❓ Question: Avons-nous déjà une base de données vectorielle configurée ?


🤖 TRAITEMENT DE LA REQUÊTE

🔍 Recherche de 3 épisodes pertinents...
✅ 3 épisode(s) récupéré(s)

📚 Mémoires utilisées:
  1. [agent] Nous avons intégré Pinecone hier pour accélérer la recherche... (score: 0.0969)
  2. [user] L'utilisateur travaille sur un projet de RAG pour l'éducatio... (score: 0.0831)
  3. [agent] Q: Quel framework ai-je dit que j'aimais, et qu'avons-nous i... (score: 0.061)

💭 Génération de la réponse...
✅ Réponse générée et enregistrée comme épisode


--------------------------------------------------------------------------------
💬 RÉPONSE DE L'AGENT
--------------------------------------------------------------------------------
Oui, nous avons intégré Pinecone hier pour accélérer la recherche s

### 5.5 Test 4: Question sans mémoire pertinente

In [14]:
question4 = "Quelle est la capitale de la France ?"

print("\n" + "#"*80)
print("TEST 4: Question sans mémoire pertinente")
print("#"*80)
print(f"\n❓ Question: {question4}\n")

answer4, used_mems4 = agent_respond(question4)

print("\n" + "-"*80)
print("💬 RÉPONSE DE L'AGENT")
print("-"*80)
print(answer4)

print("\n" + "-"*80)
print("📚 MÉMOIRES UTILISÉES")
print("-"*80)
if used_mems4:
    for i, m in enumerate(used_mems4, 1):
        print(f"\n{i}. {m['when']} :: {m['summary'][:80]}...")
        print(f"   Score: {m['score']}")
else:
    print("Aucune mémoire pertinente trouvée.")


################################################################################
TEST 4: Question sans mémoire pertinente
################################################################################

❓ Question: Quelle est la capitale de la France ?


🤖 TRAITEMENT DE LA REQUÊTE

🔍 Recherche de 3 épisodes pertinents...
✅ 3 épisode(s) récupéré(s)

📚 Mémoires utilisées:
  1. [agent] Q: Quel framework ai-je dit que j'aimais, et qu'avons-nous i... (score: -0.2192)
  2. [system] Épisode de test pour vérifier le fonctionnement du système... (score: -0.2225)
  3. [agent] Q: Avons-nous déjà une base de données vectorielle configuré... (score: -0.2536)

💭 Génération de la réponse...
✅ Réponse générée et enregistrée comme épisode


--------------------------------------------------------------------------------
💬 RÉPONSE DE L'AGENT
--------------------------------------------------------------------------------
La capitale de la France est Paris.

--------------------------------------------

### 5.6 Visualisation de toute la mémoire

In [15]:
print("\n" + "="*80)
print("📚 CONTENU COMPLET DE LA MÉMOIRE ÉPISODIQUE")
print("="*80)
print(f"\n🔢 Total: {episodes.count()} épisodes\n")

# Récupérer tous les épisodes
all_episodes = episodes.get(include=["metadatas", "documents"])

if all_episodes["ids"]:
    for i, eid in enumerate(all_episodes["ids"], 1):
        meta = all_episodes["metadatas"][i-1]
        doc = all_episodes["documents"][i-1]

        print(f"\n{'-'*80}")
        print(f"Épisode {i}")
        print(f"{'-'*80}")
        print(f"🆔 ID: {eid}")
        print(f"👤 Qui: {meta['who']}")
        print(f"📅 Quand: {ts_to_str(int(meta['ts']))}")
        print(f"🏷️  Tags: {meta.get('tags', 'Aucun')}")
        print(f"📝 Contenu: {doc}")
else:
    print("Aucun épisode dans la mémoire.")


📚 CONTENU COMPLET DE LA MÉMOIRE ÉPISODIQUE

🔢 Total: 10 épisodes


--------------------------------------------------------------------------------
Épisode 1
--------------------------------------------------------------------------------
🆔 ID: d4018dcd-f9d3-46ad-a68d-f2e59a7297e1
👤 Qui: system
📅 Quand: 2026-01-30 11:09:57 UTC
🏷️  Tags: test,init
📝 Contenu: Épisode de test pour vérifier le fonctionnement du système

--------------------------------------------------------------------------------
Épisode 2
--------------------------------------------------------------------------------
🆔 ID: 2b2bed46-3f4e-469f-9559-3b76b32ff633
👤 Qui: user
📅 Quand: 2026-01-30 11:14:30 UTC
🏷️  Tags: pref,framework,langchain,education
📝 Contenu: L'utilisateur a dit que son framework préféré est LangChain et qu'il construit des agents pour l'éducation.

--------------------------------------------------------------------------------
Épisode 3
---------------------------------------------------------------

---
## 6. Enrichissement avec Rôles & Tags (Optionnel)
**Durée:** 10–15 minutes

Amélioration du système avec:
- **Importance**: Score 1-5 pour booster les mémoires importantes
- **Filtrage par tags**: Récupérer uniquement certains types d'épisodes

### 6.1 Ajout d'un champ d'importance

In [16]:
def add_episode_enhanced(summary: str, who: str = "user", tags=None, importance: int = 3):
    """
    Version enrichie: ajoute un champ d'importance (1-5).

    Args:
        summary: Résumé de l'épisode
        who: Qui a généré l'épisode (user/agent/system)
        tags: Liste de tags pour catégorisation
        importance: Score d'importance 1-5 (1=trivial, 5=critique)
    """
    importance = max(1, min(5, importance))  # Clamp entre 1 et 5

    eid = str(uuid.uuid4())
    meta = {
        "who": who,
        "summary": summary,
        "ts": now_ts(),
        "tags": ",".join(tags) if tags else "",
        "importance": importance
    }
    episodes.add(
        ids=[eid],
        embeddings=[embed(summary)],
        metadatas=[meta],
        documents=[summary]
    )
    return eid

print("✅ Fonction add_episode_enhanced() définie")
print("📊 Niveaux d'importance:")
print("   1 = Trivial (conversation informelle)")
print("   2 = Faible (information secondaire)")
print("   3 = Moyen (information utile) [défaut]")
print("   4 = Élevé (décision, préférence importante)")
print("   5 = Critique (deadline, milestone, décision majeure)")

✅ Fonction add_episode_enhanced() définie
📊 Niveaux d'importance:
   1 = Trivial (conversation informelle)
   2 = Faible (information secondaire)
   3 = Moyen (information utile) [défaut]
   4 = Élevé (décision, préférence importante)
   5 = Critique (deadline, milestone, décision majeure)


In [17]:
def search_episodes_enhanced(query: str, k: int = 5, alpha: float = 0.7, tau_hours: float = 72,
                            importance_boost: float = 0.1):
    """
    Recherche avec boost d'importance.

    Score final = score_hybride × (1 + importance_boost × (importance - 3))

    Args:
        importance_boost: Facteur de boost par niveau d'importance (défaut 0.1 = 10%)
    """
    qemb = embed(query)

    res = episodes.query(
        query_embeddings=[qemb],
        n_results=min(20, episodes.count()),
        include=["metadatas", "distances", "documents"]
    )

    if not res["ids"] or len(res["ids"][0]) == 0:
        return []

    now = now_ts()
    tau = tau_hours * 3600.0
    out = []

    for i in range(len(res["ids"][0])):
        meta = res["metadatas"][0][i]
        doc = res["documents"][0][i]

        distance = res["distances"][0][i] if res["distances"] else 0.0
        similarity = 1 - distance

        dt = max(0, now - int(meta["ts"]))
        rec = math.exp(-dt / tau)

        blended = alpha * similarity + (1 - alpha) * rec

        # Boost basé sur l'importance
        importance = int(meta.get("importance", 3))
        importance_multiplier = 1 + importance_boost * (importance - 3)
        final_score = blended * importance_multiplier

        out.append({
            "id": res["ids"][0][i],
            "summary": doc,
            "who": meta["who"],
            "ts": meta["ts"],
            "when": ts_to_str(int(meta["ts"])),
            "similarity": round(similarity, 4),
            "recency": round(rec, 4),
            "importance": importance,
            "base_score": round(blended, 4),
            "score": round(final_score, 4),
            "tags": meta.get("tags", "")
        })

    out.sort(key=lambda x: x["score"], reverse=True)
    return out[:k]

print("✅ Fonction search_episodes_enhanced() définie")
print("📈 Le boost d'importance modifie le score final:")
print("   - Importance 5: +20% au score")
print("   - Importance 4: +10% au score")
print("   - Importance 3: Aucun changement (baseline)")
print("   - Importance 2: -10% au score")
print("   - Importance 1: -20% au score")

✅ Fonction search_episodes_enhanced() définie
📈 Le boost d'importance modifie le score final:
   - Importance 5: +20% au score
   - Importance 4: +10% au score
   - Importance 3: Aucun changement (baseline)
   - Importance 2: -10% au score
   - Importance 1: -20% au score


### 6.2 Recherche avec filtrage par tags

In [18]:
def search_episodes_with_filter(query: str, include_tags=None, exclude_tags=None,
                                k: int = 5, **kwargs):
    """
    Recherche avec filtrage côté client par tags.

    Args:
        query: Texte de recherche
        include_tags: Liste de tags - l'épisode doit avoir AU MOINS UN de ces tags
        exclude_tags: Liste de tags - l'épisode NE DOIT PAS avoir ces tags
        k: Nombre de résultats à retourner
        **kwargs: Autres paramètres passés à search_episodes_enhanced
    """
    # Récupérer plus de résultats pour compenser le filtrage
    all_results = search_episodes_enhanced(query, k=k*3, **kwargs)

    filtered = []
    for m in all_results:
        episode_tags = set(m["tags"].split(",")) if m["tags"] else set()

        # Vérifier les exclusions en premier
        if exclude_tags:
            if any(tag in episode_tags for tag in exclude_tags):
                continue

        # Vérifier les inclusions
        if include_tags:
            if not any(tag in episode_tags for tag in include_tags):
                continue

        filtered.append(m)

        if len(filtered) >= k:
            break

    return filtered

print("✅ Fonction search_episodes_with_filter() définie")
print("🏷️  Permet de filtrer par tags:")
print("   - include_tags: Inclure uniquement les épisodes avec CES tags")
print("   - exclude_tags: Exclure les épisodes avec CES tags")

✅ Fonction search_episodes_with_filter() définie
🏷️  Permet de filtrer par tags:
   - include_tags: Inclure uniquement les épisodes avec CES tags
   - exclude_tags: Exclure les épisodes avec CES tags


### 6.3 Tests des fonctionnalités enrichies

In [19]:
print("\n" + "="*80)
print("🧪 TESTS DES FONCTIONNALITÉS ENRICHIES")
print("="*80)

# Ajouter des épisodes avec différents niveaux d'importance
print("\n📝 Ajout d'épisodes test avec importance variable...\n")

eid_low = add_episode_enhanced(
    "Conversation informelle sur la météo à Paris.",
    who="user",
    tags=["small-talk", "weather"],
    importance=1
)
print(f"✅ Épisode trivial ajouté (importance=1): {eid_low[:8]}...")

eid_high = add_episode_enhanced(
    "DEADLINE CRITIQUE: Le projet doit être livré avant le 15 février 2026 pour la démo client.",
    who="user",
    tags=["deadline", "milestone", "project"],
    importance=5
)
print(f"✅ Épisode critique ajouté (importance=5): {eid_high[:8]}...")

eid_decision = add_episode_enhanced(
    "Décision prise: nous utiliserons React pour le frontend au lieu de Vue.",
    who="agent",
    tags=["decision", "frontend", "tech-stack"],
    importance=4
)
print(f"✅ Épisode décision ajouté (importance=4): {eid_decision[:8]}...")


🧪 TESTS DES FONCTIONNALITÉS ENRICHIES

📝 Ajout d'épisodes test avec importance variable...

✅ Épisode trivial ajouté (importance=1): 9b00b4d4...
✅ Épisode critique ajouté (importance=5): 6a742db3...
✅ Épisode décision ajouté (importance=4): 1b8adff6...


In [20]:
# Test 1: Recherche avec boost d'importance
print("\n" + "-"*80)
print("🔍 TEST: Recherche 'projet' avec boost d'importance")
print("-"*80)

results_boosted = search_episodes_enhanced("projet", k=5, importance_boost=0.1)

print(f"\n📊 {len(results_boosted)} résultats trouvés:\n")
for i, r in enumerate(results_boosted, 1):
    print(f"{i}. [{r['who']}] {r['summary'][:60]}...")
    print(f"   Importance: {r['importance']}/5 | Score base: {r['base_score']} → Score final: {r['score']}")
    print()


--------------------------------------------------------------------------------
🔍 TEST: Recherche 'projet' avec boost d'importance
--------------------------------------------------------------------------------

📊 5 résultats trouvés:

1. [user] DEADLINE CRITIQUE: Le projet doit être livré avant le 15 fév...
   Importance: 5/5 | Score base: 0.1782 → Score final: 0.2138

2. [user] L'utilisateur travaille sur un projet de RAG pour l'éducatio...
   Importance: 3/5 | Score base: 0.1271 → Score final: 0.1271

3. [agent] Q: Rappelle-moi ce que nous avons discuté à propos des boucl...
   Importance: 3/5 | Score base: 0.0418 → Score final: 0.0418

4. [agent] Q: Quel framework ai-je dit que j'aimais, et qu'avons-nous i...
   Importance: 3/5 | Score base: 0.0128 → Score final: 0.0128

5. [agent] Décision prise: nous utiliserons React pour le frontend au l...
   Importance: 4/5 | Score base: 0.0068 → Score final: 0.0075



In [21]:
# Test 2: Filtrage par tags (inclure uniquement les décisions)
print("\n" + "-"*80)
print("🔍 TEST: Recherche avec filtre include_tags=['decision', 'milestone']")
print("-"*80)

results_filtered = search_episodes_with_filter(
    "projet",
    include_tags=["decision", "milestone"],
    k=5
)

print(f"\n📊 {len(results_filtered)} résultats filtrés:\n")
for i, r in enumerate(results_filtered, 1):
    print(f"{i}. [{r['who']}] {r['summary'][:60]}...")
    print(f"   Tags: {r['tags']} | Score: {r['score']}")
    print()


--------------------------------------------------------------------------------
🔍 TEST: Recherche avec filtre include_tags=['decision', 'milestone']
--------------------------------------------------------------------------------

📊 2 résultats filtrés:

1. [user] DEADLINE CRITIQUE: Le projet doit être livré avant le 15 fév...
   Tags: deadline,milestone,project | Score: 0.2137

2. [agent] Décision prise: nous utiliserons React pour le frontend au l...
   Tags: decision,frontend,tech-stack | Score: 0.0076



In [22]:
# Test 3: Filtrage par exclusion (exclure small-talk)
print("\n" + "-"*80)
print("🔍 TEST: Recherche avec filtre exclude_tags=['small-talk', 'test']")
print("-"*80)

results_excluded = search_episodes_with_filter(
    "projet framework",
    exclude_tags=["small-talk", "test"],
    k=5
)

print(f"\n📊 {len(results_excluded)} résultats (sans small-talk ni test):\n")
for i, r in enumerate(results_excluded, 1):
    print(f"{i}. [{r['who']}] {r['summary'][:70]}...")
    print(f"   Tags: {r['tags']}")
    print()


--------------------------------------------------------------------------------
🔍 TEST: Recherche avec filtre exclude_tags=['small-talk', 'test']
--------------------------------------------------------------------------------

📊 5 résultats (sans small-talk ni test):

1. [agent] Q: Quel framework ai-je dit que j'aimais, et qu'avons-nous intégré hie...
   Tags: dialog,qa

2. [user] L'utilisateur a dit que son framework préféré est LangChain et qu'il c...
   Tags: pref,framework,langchain,education

3. [user] DEADLINE CRITIQUE: Le projet doit être livré avant le 15 février 2026 ...
   Tags: deadline,milestone,project

4. [agent] Q: Rappelle-moi ce que nous avons discuté à propos des boucles de plan...
   Tags: dialog,qa

5. [user] L'utilisateur travaille sur un projet de RAG pour l'éducation avec Chr...
   Tags: project,rag,education,chromadb



---
## 7. Intégration dans des Agents Avancés
**Durée:** Lecture et application future

Cette section explique comment intégrer la mémoire épisodique dans des architectures d'agents plus complexes.

### 7.1 Stratégies d'intégration

#### **A. Préfixe de contexte automatique**
```python
def build_system_prompt_with_memory(base_prompt: str, user_query: str, k: int = 3):
    """Ajoute automatiquement les épisodes pertinents au prompt système."""
    memories = search_episodes(user_query, k=k)
    memory_context = format_memories(memories)
    
    return f"""{base_prompt}

MÉMOIRE ÉPISODIQUE:
{memory_context}

Utilise ces informations pour contextualiser ta réponse.
"""
```

#### **B. Logging des résultats d'outils**
```python
def execute_tool_with_memory(tool_name: str, params: dict):
    """Exécute un outil et enregistre le résultat comme épisode."""
    result = execute_tool(tool_name, params)
    
    # Enregistrer le succès/échec
    if result.success:
        add_episode_enhanced(
            f"Outil {tool_name} exécuté avec succès: {result.summary}",
            who="agent",
            tags=["tool", tool_name, "success"],
            importance=3
        )
    else:
        add_episode_enhanced(
            f"Échec de {tool_name}: {result.error}",
            who="agent",
            tags=["tool", tool_name, "failure"],
            importance=4  # Les échecs sont importants pour l'apprentissage
        )
    
    return result
```

#### **C. Intégration avec RAG**
```python
def hybrid_retrieval(query: str, k_docs: int = 5, k_episodes: int = 3):
    """Combine récupération documentaire (RAG) et mémoire épisodique."""
    # Récupération documentaire classique
    docs = retrieve_documents(query, k=k_docs)
    
    # Récupération d'épisodes pertinents
    episodes = search_episodes(query, k=k_episodes)
    
    context = f"""
DOCUMENTS DE RÉFÉRENCE:
{format_documents(docs)}

EXPÉRIENCE PASSÉE (Mémoire épisodique):
{format_memories(episodes)}
"""
    return context
```

#### **D. Apprentissage des préférences utilisateur**
```python
def detect_and_store_preferences(user_message: str):
    """Détecte et enregistre les préférences utilisateur."""
    # Patterns de préférences
    preference_patterns = [
        r"j'aime|je préfère|je veux",
        r"mon.*préféré",
        r"je déteste|je n'aime pas"
    ]
    
    for pattern in preference_patterns:
        if re.search(pattern, user_message, re.IGNORECASE):
            add_episode_enhanced(
                user_message,
                who="user",
                tags=["preference", "user-profile"],
                importance=4
            )
            break
```

### 7.2 Cas d'usage avancés

#### **1. Agent avec planification**
- Enregistrer chaque étape du plan comme épisode
- Récupérer les plans similaires passés pour améliorer la planification actuelle
- Tags: `["planning", "step-1", "success/failure"]`

#### **2. Agent multi-sessions**
- Persistance de la mémoire entre sessions
- Résumé automatique des sessions passées
- Tags: `["session-{id}", "summary"]`

#### **3. Agent collaboratif**
- Mémoire partagée entre plusieurs agents
- Tracking de qui a fait quoi
- Tags: `["agent-{name}", "collaboration"]`

#### **4. Agent apprenant**
- Enregistrer les feedbacks utilisateurs
- Récupérer les erreurs passées pour éviter répétition
- Tags: `["feedback", "error", "correction"]`

### 7.3 Exemple complet: Agent RAG avec mémoire

In [23]:
def rag_agent_with_memory(user_query: str, document_collection, k_docs: int = 3, k_episodes: int = 3):
    """
    Agent RAG enrichi avec mémoire épisodique.

    Workflow:
    1. Récupère documents pertinents (RAG)
    2. Récupère épisodes pertinents (mémoire)
    3. Fusionne les deux sources de contexte
    4. Génère réponse
    5. Enregistre l'interaction
    """
    print(f"\n🔍 Recherche RAG + Mémoire pour: '{user_query}'\n")

    # 1. Récupération documentaire (simulée ici)
    print(f"📚 Récupération de {k_docs} documents...")
    # docs = document_collection.query(user_query, k=k_docs)
    docs = ["[Document simulé 1]", "[Document simulé 2]"]  # Simulation
    print(f"✅ {len(docs)} documents récupérés")

    # 2. Récupération épisodes
    print(f"🧠 Récupération de {k_episodes} épisodes...")
    episodes_mem = search_episodes_enhanced(user_query, k=k_episodes)
    print(f"✅ {len(episodes_mem)} épisodes récupérés")

    # 3. Construction du contexte hybride
    doc_context = "\n".join(f"- {doc}" for doc in docs)
    memory_context = format_memories(episodes_mem)

    system_prompt = f"""
Tu es un assistant RAG avec mémoire épisodique.

DOCUMENTS DE RÉFÉRENCE:
{doc_context}

EXPÉRIENCE PASSÉE (Mémoire épisodique):
{memory_context}

Instructions:
- Priorise les documents de référence pour les faits objectifs
- Utilise la mémoire épisodique pour le contexte personnel et les préférences
- Cite tes sources quand possible
- Réponds en français de manière concise et précise
"""

    # 4. Génération de réponse
    print("\n💭 Génération de la réponse...")
    answer = llm_chat(system_prompt, user_query)

    # 5. Enregistrement
    add_episode_enhanced(
        f"RAG Query: {user_query} | Answer: {answer[:200]}",
        who="agent",
        tags=["rag", "query", "answered"],
        importance=3
    )
    print("✅ Interaction enregistrée\n")

    return {
        "answer": answer,
        "documents_used": len(docs),
        "episodes_used": episodes_mem
    }

print("✅ Fonction rag_agent_with_memory() définie")

✅ Fonction rag_agent_with_memory() définie


In [24]:
# Test de l'agent RAG avec mémoire
result = rag_agent_with_memory(
    "Quel framework devrais-je utiliser pour mon projet éducatif ?",
    document_collection=None,  # Collection simulée
    k_docs=3,
    k_episodes=3
)

print("="*80)
print("💬 RÉPONSE DE L'AGENT RAG")
print("="*80)
print(result["answer"])

print("\n" + "-"*80)
print("📊 MÉTADONNÉES")
print("-"*80)
print(f"Documents utilisés: {result['documents_used']}")
print(f"Épisodes utilisés: {len(result['episodes_used'])}")

if result['episodes_used']:
    print("\n🧠 Détail des épisodes:")
    for i, ep in enumerate(result['episodes_used'], 1):
        print(f"  {i}. {ep['summary'][:60]}... (score: {ep['score']})")


🔍 Recherche RAG + Mémoire pour: 'Quel framework devrais-je utiliser pour mon projet éducatif ?'

📚 Récupération de 3 documents...
✅ 2 documents récupérés
🧠 Récupération de 3 épisodes...
✅ 3 épisodes récupérés

💭 Génération de la réponse...
✅ Interaction enregistrée

💬 RÉPONSE DE L'AGENT RAG
Vous avez mentionné que votre framework préféré est LangChain, ce qui pourrait être un excellent choix pour votre projet éducatif. Étant donné que vous travaillez également sur un projet de RAG (Récupération-Augmentation de Génération) avec ChromaDB et OpenAI, LangChain pourrait s'intégrer efficacement dans votre flux de travail.

--------------------------------------------------------------------------------
📊 MÉTADONNÉES
--------------------------------------------------------------------------------
Documents utilisés: 2
Épisodes utilisés: 3

🧠 Détail des épisodes:
  1. L'utilisateur a dit que son framework préféré est LangChain ... (score: 0.3306)
  2. L'utilisateur travaille sur un projet de 

---
## 🎯 Résumé et Points Clés

### Ce que vous avez appris:

1. **Mémoire épisodique**: Stockage d'interactions avec métadonnées (qui/quoi/quand)
2. **Embeddings**: Représentation vectorielle pour recherche sémantique
3. **Score hybride**: Combinaison similarité + récence avec décroissance exponentielle
4. **ChromaDB**: Base de données vectorielle simple et efficace
5. **Enrichissement**: Importance, tags, filtrage pour améliorer la pertinence
6. **Intégration**: Comment incorporer dans des agents RAG et planificateurs

### Formules importantes:

**Score hybride de base:**
```
score = α·similarity + (1-α)·recency
recency = exp(-Δt/τ)
```

**Score avec importance:**
```
final_score = score × [1 + β·(importance - 3)]
```

### Paramètres typiques:
- **α (alpha)**: 0.7 (70% similarité, 30% récence)
- **τ (tau)**: 72h (temps de demi-vie)
- **β (beta)**: 0.1 (boost 10% par niveau d'importance)
- **k**: 3-5 (nombre d'épisodes à récupérer)

### Prochaines étapes:
1. Expérimenter avec différentes valeurs de α et τ
2. Ajouter un système de résumé automatique des épisodes
3. Implémenter un mécanisme d'oubli (purge des vieux épisodes)
4. Intégrer avec vos propres agents et workflows
5. Tester avec des données réelles de vos projets

---
## 📝 Exercices Pratiques (Optionnel)

### Exercice 1: Ajustement des paramètres
Modifiez α et τ pour différents cas d'usage:
- Support client: α=0.5, τ=24h (équilibre, récence importante)
- Assistant personnel: α=0.8, τ=168h (similarité prioritaire, mémoire longue)
- Chatbot conversationnel: α=0.3, τ=2h (récence forte, court terme)

### Exercice 2: Système de notation
Implémenter un système où l'utilisateur peut noter les réponses (1-5 étoiles) et ajuster l'importance des épisodes en fonction.

### Exercice 3: Résumé automatique
Créer une fonction qui résume automatiquement les N derniers épisodes en un seul épisode "résumé de session".

### Exercice 4: Dashboard de mémoire
Créer une fonction de visualisation montrant:
- Distribution des tags
- Timeline des épisodes
- Statistiques d'importance
- Qui (user/agent) a généré le plus d'épisodes

In [ ]:
# ESPACE POUR VOS EXPÉRIMENTATIONS
# Utilisez cette cellule pour tester vos propres idées

# Exemple: Tester différentes valeurs de α
# for alpha_val in [0.3, 0.5, 0.7, 0.9]:
#     print(f"\nAlpha = {alpha_val}")
#     results = search_episodes("votre query", k=3, alpha=alpha_val)
#     # Analyser les résultats...

pass

---
## 🔧 Utilitaires & Debug

In [ ]:
def reset_memory():
    """⚠️ ATTENTION: Efface TOUTE la mémoire épisodique!"""
    global episodes, chroma
    try:
        chroma.delete_collection(name="episodic_memory")
    except:
        pass
    episodes = chroma.get_or_create_collection(name="episodic_memory")
    print("🗑️ Mémoire épisodique réinitialisée")
    print(f"🔢 Nombre d'épisodes: {episodes.count()}")

def memory_stats():
    """Affiche des statistiques sur la mémoire."""
    all_eps = episodes.get(include=["metadatas"])

    if not all_eps["ids"]:
        print("❌ Mémoire vide")
        return

    total = len(all_eps["ids"])

    # Comptage par qui
    who_counts = {}
    tag_counts = {}
    importance_counts = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}

    for meta in all_eps["metadatas"]:
        # Who
        who = meta["who"]
        who_counts[who] = who_counts.get(who, 0) + 1

        # Tags
        tags = meta.get("tags", "").split(",")
        for tag in tags:
            if tag:
                tag_counts[tag] = tag_counts.get(tag, 0) + 1

        # Importance
        imp = int(meta.get("importance", 3))
        importance_counts[imp] = importance_counts.get(imp, 0) + 1

    print("\n" + "="*80)
    print("📊 STATISTIQUES DE LA MÉMOIRE ÉPISODIQUE")
    print("="*80)
    print(f"\n🔢 Total d'épisodes: {total}")

    print("\n👥 Distribution par acteur:")
    for who, count in sorted(who_counts.items(), key=lambda x: x[1], reverse=True):
        pct = (count/total)*100
        print(f"  {who}: {count} ({pct:.1f}%)")

    print("\n🏷️  Top 10 tags:")
    for tag, count in sorted(tag_counts.items(), key=lambda x: x[1], reverse=True)[:10]:
        print(f"  {tag}: {count}")

    print("\n⭐ Distribution importance:")
    for imp in range(1, 6):
        count = importance_counts[imp]
        pct = (count/total)*100 if total > 0 else 0
        bar = "█" * int(pct/2)
        print(f"  {imp}: {bar} {count} ({pct:.1f}%)")

def search_by_tags_only(tags: list, limit: int = 10):
    """Recherche des épisodes uniquement par tags (sans query sémantique)."""
    all_eps = episodes.get(include=["metadatas", "documents"])

    if not all_eps["ids"]:
        return []

    results = []
    for i, eid in enumerate(all_eps["ids"]):
        meta = all_eps["metadatas"][i]
        episode_tags = set(meta.get("tags", "").split(","))

        if any(tag in episode_tags for tag in tags):
            results.append({
                "id": eid,
                "summary": all_eps["documents"][i],
                "who": meta["who"],
                "when": ts_to_str(int(meta["ts"])),
                "tags": meta.get("tags", ""),
                "importance": int(meta.get("importance", 3))
            })

            if len(results) >= limit:
                break

    return results

print("✅ Fonctions utilitaires chargées:")
print("   - reset_memory(): Efface toute la mémoire")
print("   - memory_stats(): Affiche les statistiques")
print("   - search_by_tags_only(tags): Recherche par tags uniquement")

In [ ]:
# Afficher les statistiques de la mémoire
memory_stats()

---
## 🎓 Conclusion

Félicitations! Vous avez implémenté un système complet de mémoire épisodique pour agents.

**Ce lab vous a permis de:**
- ✅ Comprendre les embeddings et la recherche vectorielle
- ✅ Implémenter un scoring hybride similarité/récence
- ✅ Construire un agent avec mémoire contextuelle
- ✅ Enrichir avec importance et tags
- ✅ Intégrer dans des architectures avancées (RAG, planning)

**Pour aller plus loin:**
- Explorez d'autres bases vectorielles (Pinecone, Weaviate, Qdrant)
- Implémentez un système de mémoire sémantique (faits généraux) vs épisodique (expériences)
- Ajoutez une mémoire procédurale (comment faire les choses)
- Créez des agents multi-mémoires avec différentes stratégies de récupération

**Ressources:**
- [ChromaDB Documentation](https://docs.trychroma.com/)
- [OpenAI Embeddings Guide](https://platform.openai.com/docs/guides/embeddings)
- [LangChain Memory](https://python.langchain.com/docs/modules/memory/)

---

**Questions? Suggestions? Feedback?**  
N'hésitez pas à expérimenter avec le code et adapter à vos cas d'usage!

🚀 **Happy coding!**